In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Lire le datset de (Valenzuela et al, 2015)
dataset=pd.read_csv('../input/mydata/ValenzuelaAnnotations.csv')

#Visualiser notre dataset
dataset.head(10)

In [ ]:
# Récupérer les codes d'articles présents dans le dataset (les columns Paper et Cited-by)
union = pd.concat([dataset['Paper'], dataset['Cited-by']])
union=union.drop_duplicates()
#print(union.values)

In [ ]:
# web scrapping pour recuperer les liens des fichiers PDF pour les telecharger par la suite

import re
import requests
from bs4 import BeautifulSoup

# Récupérer l'URL, l'auteur et l'année de l'article à partir de son code de l'anthologie ACL

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

url="https://www.aclweb.org/anthology/"
r1 = requests.get(url, headers=headers)
coverpage = r1.content
soup1 = BeautifulSoup(coverpage, 'html.parser')
#print(str(soup1))
url_list =[]
author_list =[]
years_list =[]
for i in range(len(union.values)):
    r1 = requests.get(url+str(union.values[i]), headers=headers)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'html.parser')
    #z = re.findall(r'url = \"(.+)\"',str(soup1))
    urls=re.findall(r'url =( &quot;| \")(.+)(&quot|\")',str(soup1))
    if len(urls)!=0: url_list.append(urls[0][1])
    else: print(union.values[[i]])
    authors=re.findall(r'<meta content=\"([a-zA-Zéłńčøš\. ]+)\" name="citation_author"/>',str(soup1))
    if len(authors)!=0:author_list.append(authors)
    else: print(union.values[[i]])
    year = re.findall(r'year =( &quot;| \")([\d]+)(&quot;|\")', str(soup1))
    if len(year)!=0: years_list.append(year[0][1])
    else: print(union.values[[i]])

In [ ]:
#L'article dont le code a été affiché précédemment [W07-2058] n'a pas pu être accédée 
#car le fichier n'existe pas dans l'anthologie ACL

In [ ]:
#Afficher nombre d'articles unique auxquels on a pu accéder
print(len(url_list))
print(len(author_list))
print(len(years_list))

In [ ]:
# Télécharger les articles  auxquels on a pu accéder précédemment
import urllib

for url in url_list:
    if '.pdf' not in url:
          url=url+'.pdf'
    try:
      urllib.request.urlretrieve(url, url[-12:])
    except :
      print("Variable "+ str(url) +" is not defined")
      pass

In [ ]:
import re
import requests
from bs4 import BeautifulSoup

# Fonction pour récupérer l'auteur et l'année de l'article
def get_authors_year(code):

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
    }
    # Récupérer les articles de l'anthologie ACL
    url="https://www.aclweb.org/anthology/"
    r1 = requests.get(url+str(code), headers=headers)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'html.parser')
    years_list =[]
    author_list =[]
    soup1 = BeautifulSoup(coverpage, 'html.parser')
    authors=re.findall(r'<meta content=\"([a-zA-Zéłńčøš\. ]+)\" name="citation_author"/>',str(soup1))
    
    if len(authors)!=0:
        author_list.append(authors)
    else: 
        print('author non found')
    
    year = re.findall(r'year =( &quot;| \")([\d]+)(&quot;|\")', str(soup1))
    if len(year)!=0: 
        years_list.append(year[0][1])
    else: 
        print('year non found')


    return author_list[0],years_list[0]

In [ ]:
#Exemple d'exécution de la fonction get_authors_year
print(((get_authors_year('D07-1009'))))


In [ ]:
#Après avoir analysé notre ensemble d'articles, on a remarqué que certains fichiers se convertissent mal 
#on a récupéré leurs codes dans la liste suivante
removed_files = ['N06-1038','D08-1070','D12-1092','E09-1091','P12-1076','W10-2923','W10-2420','P00-1061','D09-1057','S13-1014','P00-1061',
                 'P11-2123','P09-2030','E06-1011','W07-2058','L08-1267','L08-1328','L08-1584','E06-1012',
                 'D09-1057','W11-0112','I08-6004','S13-1021','W13-3616','I08-4013','D09-1016','W11-0135','D07-1019','W08-0504','D13-1159']

In [ ]:
#Supprimer les articles dont le code est dans la liste removed_files
df = dataset[~dataset['Cited-by'].isin(removed_files)]
df = df.reset_index(drop=True)
print("Nombre d'articles apres la suppression : "+str(len(df)))


In [ ]:
#Fonction pour recuperer le contenu des articles sans la partie abstract et sans la partie des references 
# pour garder le contenu essentielle
def get_pdf_content(string):
    
    if re.search('(\nintroduction\s*?\n)',string,re.I):
        deb=re.search('(\nintroduction\s*?\n)',string,re.I).span()[0]
    elif re.search('(\nabstract\s*?\n)',string,re.I): 
        deb=re.search('(\nabstract\s*?\n)',string,re.I).span()[0]  
    else: 
        deb = 0  
      
   
    if re.search('(\n\s*s?ecnerefer\n)',string[::-1],re.I):
        fin = len(string) - re.search('(\n\s*?s?ecnerefer\n)',string[::-1],re.I).span()[0]
    else : 
        fin = len(string)

    return string[deb:fin]


In [ ]:
#pour la conversion des pdf vers text on a utilsé la bibliotheque tika
#apres on a creer une fonction pour nettoyer le texte du entetes et figures et tableux et les urls
def clean_pdf(all_text):
    #supprimer les tableaux et figures
    regex_for_num =r'(\d{1,}[.,]?\d{1,}%?|\b[a-zA-Z]{1,3}\b)'
    regex_for_urls =r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    list_paragraphs = re.split(r'\n\n',all_text)  
    
    for i in range(len(list_paragraphs)):
        if re.search(r'^((Figure|Table) \d{1,2}[:.]\s+?[A-Z])',list_paragraphs[i]):
            #print('haha')
            list_paragraphs[i]=''
            list_paragraphs[i-1]=''
        list_paragraphs[i] = re.sub(regex_for_urls,'',list_paragraphs[i])
      
    
    cleaned = [paragraph for paragraph in list_paragraphs if len(paragraph)>70]
    
    txt = '\n\n'.join(cleaned)
    return txt

In [ ]:
# Fonction pour extraire d'un article les citations et leur contexte qui contient 3 phrases 
import re
from tika import parser
import signal
class TimeoutException(Exception):   # Custom exception class
    pass

def timeout_handler(signum, frame):   # Custom signal handler
    raise TimeoutException

# Change the behavior of SIGALRM
signal.signal(signal.SIGALRM, timeout_handler)

#Foonction pour extraire les citations et leurs contextes (3 phrases)
def get_docsss_cita(document_code):    
    
    parsed_pdf = parser.from_file(document_code+'.pdf')
    bana = re.split(r'\n\n',clean_pdf(get_pdf_content(parsed_pdf['content'])))
    
    sentences = re.findall(r'(([A-Z])(.|\n)+?(\.)(?=(\s*[A-Z]|$)))',clean_pdf(get_pdf_content(parsed_pdf['content'])))
    sentences = [sen[0] for sen in sentences]

    pattern1 = "([([](e.g[,.]+\s)?[A-Z][^\s0-9]+(page?|-|\d{2}(\d{2})?[a-z]?|[;,]|etc|with|&|avec|et|and|al|[A-Z][^\s0-9]+|\.|\s|-\s+[^A-Z\s0-9]+)*\d{2}(\d{2})?[a-z]?(pages?\s\d{1,4}-?\d{1,4}?|-|\s|:|,|;|\d{4}|\d{2})*(etc|(among|and)\sothers)?[])])"
    pattern2 = "([A-Z][^\s0-9,;]+(etc|with|&|avec|et|and|al|[A-Z][^\s0-9]+|\.|\s|-\s+[^A-Z\s0-9]+)*?\(\d{2}(\d{2})?[a-z]?(pages?\s\d{1,4}-?\d{1,4}?|-|\s|:|,|;|\d{4}|\d{2})*(etc|(among|and)\sothers)?\))"
    pattern3 = "\[\d{1,2}\]"
    all_cita = []
    all_con = []

    for sen_index,sentence in zip(range(len(sentences)),sentences):
        sen_contex=''
        signal.alarm(1)
        try :
            if sen_index <1:
                sen_contex =sentences[sen_index]+' '+sentences[sen_index+1]
            elif sen_index == len(sentences)-1:
                sen_contex =sentences[sen_index-1]+' '+sentences[sen_index]
            else:
                #print(sen_index,len(sentences))
                sen_contex =sentences[sen_index-1]+' '+sentences[sen_index]+' '+sentences[sen_index+1]    
            
            hamida=re.findall(pattern1,re.sub('\s*\n\s*',' ', re.sub('-\s*\n\s*','',sentence)))
            style_1 = [i[0] for i in hamida]
            hamid= re.findall(pattern2,re.sub('\s*\n\s*',' ', re.sub('-\s*\n\s*','',sentence)))
            style_2 = [i[0] for i in hamid]
            salim= re.findall(pattern3,re.sub('\s*\n\s*',' ', re.sub('-\s*\n\s*','',sentence)))
            style_3 = [i[0] for i in salim]
            sen_cita = style_1+style_2+style_3
            context = [re.sub('\s*\n\s*',' ', re.sub('-\s*\n\s*','',sen_contex)) for i in sen_cita]
    
            all_cita = all_cita +sen_cita
            all_con = all_con + context
    
        except TimeoutException:
            continue # continue the for loop if function A takes more than 5 second
        else:
            # Reset the alarm
            signal.alarm(0)
    
        data_frame = pd.DataFrame(list(zip(all_cita, all_con)),
                                  columns=['Citation','Context'])
    return data_frame

#voici les citations avec des differents styles et leurs contexte d'un article W11-810
get_docsss_cita("W11-0810")

In [ ]:
#Construction de notre dataset
counter = 0
citations_list = []
Context_list = []
files_no_citations = []
files_time_prblm = []


for i in range(len(df)):

    authors,year = get_authors_year(df['Paper'][i])
    data_frame = get_docsss_cita(df['Cited-by'][i])
    
    for j in range(len(data_frame)):
            if re.search(authors[0].split()[-1] , re.sub('-?\s?\n','',data_frame['Citation'][j]),re.I ) and (year in data_frame['Citation'][j] or year[2:4] in data_frame['Citation'][j]) :
                citations_list.append(data_frame['Citation'][j])
                Context_list.append(data_frame['Context'][j])
                #print(j)
                break
    else:
        citations_list.append(None)
        Context_list.append(None)
        counter+=1


In [ ]:
# creer une fonction pour supprimer les noms des entites
import spacy

def remove_NE(text_data):
    
    nlp = spacy.load('en_core_web_sm')
    document = nlp(text_data)

    text_no_namedentities = []
    ents = [e.text for e in document.ents]
    
    for item in document:
        if item.text in ents:
            pass
        else:
            text_no_namedentities.append(item.text)
    return " ".join(text_no_namedentities)
    

In [ ]:
#Exemple d'exécution de la fonction remove_NE
text_data = 'Mohamed has achieved good results using Stanford POS tagger'
remove_NE(text_data)


In [ ]:
#Fonction pour nettoyer le contexte  
import nltk
def clean_context(pd_frame):
    list_clean = []
    pattern1 = "([([](e.g[,.]+\s)?[A-Z][^\s0-9]+(page?|-|\d{2}(\d{2})?[a-z]?|[;,]|etc|with|&|avec|et|and|al|[A-Z][^\s0-9]+|\.|\s|-\s+[^A-Z\s0-9]+)*\d{2}(\d{2})?[a-z]?(pages?\s\d{1,4}-?\d{1,4}?|-|\s|:|,|;|\d{4}|\d{2})*(etc|(among|and)\sothers)?[])])"
    pattern2 = "([A-Z][^\s0-9,;]+(etc|with|&|avec|et|and|al|[A-Z][^\s0-9]+|\.|\s|-\s+[^A-Z\s0-9]+)*?\(\d{2}(\d{2})?[a-z]?(pages?\s\d{1,4}-?\d{1,4}?|-|\s|:|,|;|\d{4}|\d{2})*(etc|(among|and)\sothers)?\))"
    
    for i in range(len(pd_frame)):
        list_clean.append(pd_frame['Context'][i].replace(pd_frame['Citation'][i],' '))
        #Supprimer les citations
        list_clean[i]=re.sub(pattern1,'', list_clean[i])
        list_clean[i]=re.sub(pattern2,'', list_clean[i])
        #Supprimer ce qui se trouve entre parenthèses et entre crochets
        list_clean[i]=re.sub("([\[\(][^\(\)\[\]]*?[\]\)])",' ', list_clean[i])
        # part-of-speech ---> part of speech
        list_clean[i]= re.sub('-',' ',list_clean[i])
        #Supprimer les mots qui ont des caractères spéciaux et des         
        list_clean[i]=' '.join([i for i in  nltk.word_tokenize(list_clean[i]) if not re.search("[^a-zA-Z]+",i)  and len(re.findall(r'[A-Z]',i)) < len(re.findall(r'[a-z]',i)) and len(i)>1])
        #Supprimer les entités nommées 
        list_clean[i] = remove_NE(list_clean[i])
    return list_clean
print('Context avant nettoyage :')
print(get_docsss_cita("W11-0810")['Context'][3])
print('\n\n')
print('Context apres nettoyage :')
print(clean_context(get_docsss_cita("W11-0810"))[3])

In [ ]:
df2= df

df2['Citation'] = citations_list
df2['Context'] = Context_list

df2 = df2.dropna()
df2= df2.reset_index(drop=True)
print(len(df2))
clean_con = clean_context(df2)
df2['Cleaned_context'] = clean_con

In [ ]:
#ceci une autre liste des articles qui se sont mal convertie
remove2 = ['P02-1057', 'C04-1077', 'P01-1008','P05-1006','P10-1036', 'W03-0424', 'P02-1055', 'N13-1059','P06-1089', 'N10-1072', 'P12-3013']

df2 = df2[~df2['Cited-by'].isin(remove2)]
df2 = df2.reset_index(drop=True)
#print(len(df2))



In [ ]:
#Visualisation du partie du notre dataset contexte
#on remarque qu'ils sont bien nettoyé du bruit
for i in range(240,245):
    print(df2['Cleaned_context'][i]+'\n\n')

In [ ]:
#Reconnaitre les citations étiquetées 2 et 3 mais qui n'existent pas dans notre dataset
for i in range(len(dataset)):
    if dataset['Follow-up'][i]>1:
        for j in range(len(df2)):
            if dataset['Cited-by'][i]==df2['Cited-by'][j] and dataset['Paper'][i]==df2['Paper'][j] and dataset['Annotator'][i]==df2['Annotator'][j]:
                break
            
        else : 
            print(get_authors_year(dataset['Paper'][i]))
            print(dataset['Annotator'][i]+'    '+dataset['Paper'][i]+'    '+dataset['Cited-by'][i]+'   '+str(dataset['Follow-up'][i]))  


In [ ]:
#Récupérer le dataset qu'on a construit dans un fichier Csv pour l'implementation des algorithmes
df2.to_csv('three_sentences.csv')